In [1]:
import pandas as pd
import time

Pandas dataset generation

In [ ]:
dataset = open("val_dataset.txt","r")
text = []
label = []
for line in dataset.readlines():
    if line == "\n":
        continue
    line = line.split(":")
    line = [x.strip(" \n") for x in line]
    text.append(line[0])
    label.append(line[1])
dataset = pd.DataFrame({"text": text, "label": label},index=[i+1 for i in range(len(text))])
dataset

,text,label
1,Predict the trend of working_time_avg for Lase...,predictions
2,Predict the future working_time_max for Assemb...,predictions
3,Predict the future offline_time_min for Assemb...,predictions
4,Predict the future offline_time_sum for Riveti...,predictions
5,Can you predict idle_time_avg for Laser Weldin...,predictions
...,...,...
60,Can you calculate Machine Usage Trend for Asse...,kpi_calc
61,Calculate availability for Testing Machine 1 f...,kpi_calc
62,Calculate MTBM for Riveting Machine 1,kpi_calc
63,Calculate material cost per unit for Low Capac...,kpi_calc


## Classification test

The notebook starts with the communication initializzation to _Gemini_1.5_flash_ API.<br>
The prompt classification is made using gemini with few shot learning, using an example for each classification label.

In [3]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from tqdm import tqdm
load_dotenv()

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# few shots examples
esempi = [
    {"testo": "Predict for the next month the cost_working_avg for Large Capacity Cutting Machine 2 based on last three months data", "etichetta": "predictions"},
    {"testo": "Generate a new kpi named machine_total_consumption which use some consumption kpis to be calculated", "etichetta": "new_kpi"},
    {"testo": "Compute the Maintenance Cost for the Riveting Machine 1 for yesterday", "etichetta": "kpi_calc"},
    {"testo": "Can describe cost_working_avg?", "etichetta": "kb_q"},
    {"testo": "Make a report about bad_cycles_min for Laser Welding Machine 1 with respect to last week", "etichetta": "report"},
    {"testo": "Create a dashboard to compare perfomances for different type of machines", "etichetta": "dashboard"},
]

# Few shot prompt creation
esempio_template = PromptTemplate(
    input_variables=["testo", "etichetta"],
    template="Text: {testo}\nLabel: {etichetta}\n"
)

few_shot_prompt = FewShotPromptTemplate(
    examples=esempi,
    example_prompt=esempio_template,
    suffix="Classify with one of the labels ['predictions', 'new_kpi', 'report', 'kb_q', 'dashboard','kpi_calc'] the following prompt:\nText: {text_input}\nLabel:",
    input_variables=["text_input"]
)


c:\Users\gdema\Anaconda3\envs\SA\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now we can start testing few shot learning on the dataset to fine-tune prompts

In [4]:
true_pos=0
mismatch = []
total = dataset.shape[0]
with tqdm(total=100) as bar:
    for (text,golden_label) in zip(dataset["text"],dataset["label"]):
        prompt = few_shot_prompt.format(text_input=text)
        out = llm.invoke(prompt)
        out=out.content.strip("\n")
        if out == golden_label:
            true_pos+=1
        else:
            mismatch.append(f"text = {text}\nTrue label = {golden_label}, predicted = {out}")
        time.sleep(4)
        bar.update(100/total)
        


print(f"Accuracy = {true_pos/total}")
print(f"There have been {len(mismatch)} wrong predictions:")
for i in mismatch:
    print(mismatch)


  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100.0/100 [04:41<00:00,  2.81s/it]  

Accuracy = 1.0
There have been 0 wrong predictions:
